In [202]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import scipy as sp
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold

Reading data:

In [203]:
def read_data(folder, name1, name2, name3):
    loc = "%s/%s.txt" %(folder, name1)
    train_set = pd.read_csv(loc, sep="\t")
    print "Training set has %s rows and %s coumns" %(train_set.shape[0], train_set.shape[1])
    loc = "%s/%s.txt" %(folder, name2)
    test_set = pd.read_csv(loc, sep="\t")
    print "Testing set has %s rows and %s coumns" %(test_set.shape[0], test_set.shape[1])
    loc = "%s/%s.txt" %(folder, name3)
    val_set = pd.read_csv(loc, sep="\t")
    print "Validation set has %s rows and %s coumns" %(val_set.shape[0], val_set.shape[1])
    return train_set, test_set, val_set



In [204]:
train_set, test_set, val_set = read_data("data5", "train_data", "test_data", "valid_data")

Training set has 428 rows and 25 coumns
Testing set has 142 rows and 25 coumns
Validation set has 142 rows and 25 coumns


In [205]:
train_set_np = train_set.values
test_set_np = test_set.values
val_set_np = val_set.values

In [206]:
def np_cleaner(train, test, val):
    train = train[~np.isnan(train).any(axis=1)]
    print "Training set has %s rows and %s coumns" %(train.shape[0], train.shape[1])
    test = test[~np.isnan(test).any(axis=1)]
    print "Testing set has %s rows and %s coumns" %(test.shape[0], test.shape[1])
    val = val[~np.isnan(val).any(axis=1)]
    print "Validation set has %s rows and %s coumns" %(val.shape[0], val.shape[1])
    return train, test, val

In [207]:
train_set_nb, test_set_nb, val_set_nb = np_cleaner(train_set_np, test_set_np, val_set_np)

Training set has 428 rows and 25 coumns
Testing set has 142 rows and 25 coumns
Validation set has 142 rows and 25 coumns


In [208]:
train_all, test_all, val_all = np_cleaner(train_set_np, test_set_np, val_set_np)

Training set has 428 rows and 25 coumns
Testing set has 142 rows and 25 coumns
Validation set has 142 rows and 25 coumns


In [209]:
train_set.columns

Index([u'GP', u'GS', u'MIN', u'FGM', u'FGA', u'FG%', u'3PM', u'3PA', u'3P%',
       u'FTM', u'FTA', u'FT%', u'OFF', u'DEF', u'TRB', u'AST', u'STL', u'BLK',
       u'PF', u'TOV', u'PTS', u'YR', u'POS', u'W', u'H'],
      dtype='object')

In [210]:
columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"POS":22,"W":23,"H":24} 

# Labels

In [211]:
def np_labeliser(train, test, val, col):
    train = train[:,col]
    test = test[:,col]
    val = val[:,col]
    return train, test, val

In [212]:
 train_labels, test_labels, val_labels = np_labeliser(train_set_nb, test_set_nb, val_set_nb, 22)

# Feature Selection

In [213]:
def np_featuriser(train, test, val,feature_list):
    
    train = np.delete(train,feature_list,1)
    test = np.delete(test,feature_list,1)
    val = np.delete(val,feature_list,1)
    
    return train, test, val

In [214]:
feature_list = [22]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

# Variance Threshold

In [215]:
def vt_fsel(train):
    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    sel.fit_transform(train)
    vt_list = sel.get_support()
    l_vt = []
    j = 0
    for i in vt_list:
        if i == False:
            l_vt.append(j)
            print "%s. featue name: %s" %(j, columns.keys()[columns.values().index(j)])
        j = j+1
    return l_vt    

In [216]:
list_vt = vt_fsel(train_all)

5. featue name: FG%
8. featue name: 3P%
11. featue name: FT%


In [217]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=10)
X = train_features_nb
y = train_labels
#print X[:1]
selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
#print scores
scores /= scores.max()
print scores

#for i in scores:
#scores

print "Most valuable features: "
remove = []
for i in range(0,25):
    remove.append(i)
    
j = 0
for i in scores:
    
    if i>0.20:
        print j, i, columns.keys()[columns.values().index(j)]
        remove.remove(j)
    if j == 21:
        j = j+2
    else:
        j = j+1
#for i in range(0,25)
print remove

[ 0.01470311  0.0074998   0.04611609  0.04593729  0.08357463  0.02805854
  0.30318339  0.34438477  0.31151492  0.02153433  0.00746457  0.12981515
  0.16454504  0.05304241  0.08644591  0.33475251  0.16157611  0.15386772
  0.03772989  0.05008168  0.05688609  0.00259443  0.75265577  1.        ]
Most valuable features: 
6 0.30318339431 3PM
7 0.344384768543 3PA
8 0.311514923218 3P%
15 0.334752506224 AST
23 0.752655766218 W
24 1.0 H
[0, 1, 2, 3, 4, 5, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22]


In [218]:
feature_list = [0, 1, 2, 3, 4, 6, 10, 11, 14, 19, 20, 21]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

# All Results

In [219]:
list_none = [22]
list_domain = [columns["GP"],columns["GS"],columns["MIN"],columns["FG%"],
      columns["3P%"],columns["FT%"],columns["PTS"],columns["YR"],columns["POS"],columns['3PM'],columns['FTM'],columns['FGM']]
list_uni = [0, 1, 2, 3, 4, 5, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22]

In [220]:
#feature_list = [columns["GP"],columns["GS"],columns["MIN"],columns["PTS"],columns["FG%"],
#     columns["3P%"],columns["FT%"],columns["YR"],columns["POS"],columns['3PM'],columns['FTM'],columns['FGM']]
#train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

In [221]:
def print_metrics(name, accuracy, conf_matrix):
    print "Feature selection: %s\n" %name
    print "Accuracy score: %s\n" %accuracy
    print "Confusion matrix:"
    print "\n%s" %conf_matrix
    print"\n"
    
def clf(clf, tr, tr_labels, val, val_labels):
        clf.fit(tr, tr_labels)
        
        pred = clf.predict(val)
        
        acc = metrics.accuracy_score(val_labels,pred)
        conf = metrics.confusion_matrix(val_labels, pred)
        
        return acc, conf

In [222]:
def clf_all(CLF, l_none, l_domain, l_uni, train_all, test_all, val_all):
    tr_none, ts_none, val_none = np_featuriser(train_all, test_all, val_all, l_none)
    print tr_none.shape
    tr_domain, ts_domain, val_domain = np_featuriser(train_all, test_all, val_all, l_domain)
    
    tr_uni, ts_uni, val_uni = np_featuriser(train_all, test_all, val_all, l_uni)
   
    
    #clfnb = GaussianNB()
    print "Naive Bayes\n"
    
    acc_none, conf_none = clf(CLF, tr_none, train_labels, val_none, val_labels)
    print_metrics("None", acc_none, conf_none)
    
    acc_domain, conf_domain = clf(CLF, tr_domain, train_labels, val_domain, val_labels)
    print_metrics("Domain knowledge", acc_domain, conf_domain)

    acc_uni, conf_uni = clf(CLF, tr_uni, train_labels, val_uni, val_labels)
    print_metrics("Univariate", acc_uni, conf_uni)

In [223]:
def clf_all(CLF, l_none, l_domain, l_uni, l_vt, train_all, test_all, val_all):
    tr_none, ts_none, val_none = np_featuriser(train_all, test_all, val_all, l_none)
    print tr_none.shape
    tr_domain, ts_domain, val_domain = np_featuriser(train_all, test_all, val_all, l_domain)
    
    tr_uni, ts_uni, val_uni = np_featuriser(train_all, test_all, val_all, l_uni)
    
    tr_vt, ts_vt, val_vt = np_featuriser(train_all, test_all, val_all, l_vt)
    
    #clfnb = GaussianNB()
    #print "Naive Bayes\n"
    
    acc_none, conf_none = clf(CLF, tr_none, train_labels, ts_none, test_labels)
    print_metrics("None", acc_none, conf_none)
    
    acc_domain, conf_domain = clf(CLF, tr_domain, train_labels, ts_domain, test_labels)
    print_metrics("Domain knowledge", acc_domain, conf_domain)

    acc_uni, conf_uni = clf(CLF, tr_uni, train_labels, ts_uni, test_labels)
    print_metrics("Univariate", acc_uni, conf_uni)
    
    acc_vt, conf_vt = clf(CLF, tr_vt, train_labels, ts_vt, test_labels)
    print_metrics("Variance Threshold", acc_vt, conf_vt)

In [224]:
clfnb = GaussianNB()
clf_all(clfnb, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

(428L, 24L)
Feature selection: None

Accuracy score: 0.661971830986

Confusion matrix:

[[13  0  0  0  0]
 [11 15  3  0  2]
 [ 0  3 14  0  1]
 [ 0  0  1 16 19]
 [ 0  0  0  8 36]]


Feature selection: Domain knowledge

Accuracy score: 0.725352112676

Confusion matrix:

[[13  0  0  0  0]
 [10 19  2  0  0]
 [ 0  2 16  0  0]
 [ 0  0  2 15 19]
 [ 0  0  0  4 40]]


Feature selection: Univariate

Accuracy score: 0.683098591549

Confusion matrix:

[[13  0  0  0  0]
 [ 6 21  3  0  1]
 [ 0  3 13  1  1]
 [ 0  0  3  7 26]
 [ 0  0  0  1 43]]


Feature selection: Variance Threshold

Accuracy score: 1.0

Confusion matrix:

[[13  0  0  0  0]
 [ 0 31  0  0  0]
 [ 0  0 18  0  0]
 [ 0  0  0 36  0]
 [ 0  0  0  0 44]]




In [225]:
svm = SVC()
svm.set_params(kernel='linear')
clf_all(svm, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

(428L, 24L)
Feature selection: None

Accuracy score: 0.802816901408

Confusion matrix:

[[10  3  0  0  0]
 [ 4 24  3  0  0]
 [ 1  4 13  0  0]
 [ 0  0  2 29  5]
 [ 0  0  0  6 38]]


Feature selection: Domain knowledge

Accuracy score: 0.830985915493

Confusion matrix:

[[13  0  0  0  0]
 [ 4 24  3  0  0]
 [ 1  4 13  0  0]
 [ 0  0  1 29  6]
 [ 0  0  0  5 39]]


Feature selection: Univariate

Accuracy score: 0.830985915493

Confusion matrix:

[[13  0  0  0  0]
 [ 6 22  3  0  0]
 [ 1  2 15  0  0]
 [ 0  0  2 26  8]
 [ 0  0  0  2 42]]


Feature selection: Variance Threshold

Accuracy score: 0.992957746479

Confusion matrix:

[[13  0  0  0  0]
 [ 0 31  0  0  0]
 [ 0  0 18  0  0]
 [ 0  0  0 36  0]
 [ 0  0  1  0 43]]




In [226]:
logreg = linear_model.LogisticRegression(C=1e5)
clf_all(logreg, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

(428L, 24L)
Feature selection: None

Accuracy score: 0.69014084507

Confusion matrix:

[[11  2  0  0  0]
 [ 6 17  7  1  0]
 [ 1  6  8  3  0]
 [ 0  0  2 20 14]
 [ 0  0  0  2 42]]


Feature selection: Domain knowledge

Accuracy score: 0.725352112676

Confusion matrix:

[[13  0  0  0  0]
 [ 7 19  5  0  0]
 [ 1  4  8  5  0]
 [ 0  0  3 21 12]
 [ 0  0  0  2 42]]


Feature selection: Univariate

Accuracy score: 0.640845070423

Confusion matrix:

[[13  0  0  0  0]
 [ 9 14  7  1  0]
 [ 1  5  6  6  0]
 [ 0  1  1 15 19]
 [ 0  0  0  1 43]]


Feature selection: Variance Threshold

Accuracy score: 0.880281690141

Confusion matrix:

[[13  0  0  0  0]
 [ 2 23  4  2  0]
 [ 0  2 14  2  0]
 [ 0  0  4 31  1]
 [ 0  0  0  0 44]]


